In [25]:
import pandas as pd
import fnmatch
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import matplotlib.pyplot as plt 

In [26]:
# Load the crime data from Brottsförebyggande Rådet (BRÅ).
df = pd.read_excel("crime_data_bra.xlsx", engine='openpyxl')

In [27]:
# Drop unecessary columns.
df = df.drop(["Brott"], axis=1)
df

,Region,År,Period,Antal,/100 000 inv
0,NaN,NaN,NaN,NaN,NaN
1,Gotlands län,1998.0,Dec,..,..
2,Gotlands län,2002.0,Okt,..,..
3,Gotlands län,2007.0,Nov,..,..
4,Gotlands län,2011.0,Nov,..,..
...,...,...,...,...,...
2516,Östergötlands län,2008.0,Mar,40,9
2517,Östergötlands län,2010.0,Jan,38,9
2518,Östergötlands län,2010.0,Mar,38,9
2519,Östergötlands län,2012.0,Dec,38,9


In [28]:
# Drop first row (only NaN value).
df = df.iloc[1:]

In [29]:
# Translate date string into int months.

# Use dictionary to translate months.
m = {
    "Jan": "1",
    "Feb": "2",
    "Mar": "3",
    "Apr": "4",
    "Maj": "5",
    "Jun":"6",
    "Jul":"7",
    "Aug":"8",
    "Sep":"9",
    "Okt":"10",
    "Nov":"11",
    "Dec": "12"
}

new_months = []
for month in df["Period"]:
    new_months.append(m[month])
df["Period"] = new_months
df["År"] = df["År"].apply(lambda x: int(x))

/var/folders/cp/4l5cyd252gl8gyd90gw38swr0000gn/T/ipykernel_55903/760187407.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Period"] = new_months
/var/folders/cp/4l5cyd252gl8gyd90gw38swr0000gn/T/ipykernel_55903/760187407.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["År"] = df["År"].apply(lambda x: int(x))


In [30]:
# Save the date in a combined column called 'date'.
# The format of this column should be YYYY-MM.

new_col = []
for a,p in zip(df["År"], df["Period"]):
    if int(p) < 10:
        new_col.append(str(a)+'-'+'0'+str(p))
    else:
        new_col.append(str(a)+'-'+str(p))
df["Date"]= new_col
df = df.drop(["År", "Period"], axis=1)
df.sort_values(by="Date")

/var/folders/cp/4l5cyd252gl8gyd90gw38swr0000gn/T/ipykernel_55903/4107241598.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Date"]= new_col


,Region,Antal,/100 000 inv,Date
981,Västra Götalands län,258,17,1995-01
43,Gotlands län,6,10,1995-01
240,Södermanlands län,28,11,1995-01
440,Örebro län,34,12,1995-01
2093,Jönköpings län,23,7,1995-01
...,...,...,...,...
716,Dalarnas län,42,15,2014-12
857,Kalmar län,37,16,2014-12
802,Västmanlands län,40,15,2014-12
261,Uppsala län,39,11,2014-12


In [31]:
# Check that the date is now formatted correctly.
df

,Region,Antal,/100 000 inv,Date
1,Gotlands län,..,..,1998-12
2,Gotlands län,..,..,2002-10
3,Gotlands län,..,..,2007-11
4,Gotlands län,..,..,2011-11
5,Gotlands län,..,..,2012-02
...,...,...,...,...
2516,Östergötlands län,40,9,2008-03
2517,Östergötlands län,38,9,2010-01
2518,Östergötlands län,38,9,2010-03
2519,Östergötlands län,38,9,2012-12


In [32]:
# Save the new cleaned dataset to a CSV file.
df.to_csv("crime_data_cleaned.csv")